# Passo a passo

# 1. Ler a documentação não oficial 
[Selenium with Python](https://selenium-python.readthedocs.io/)

# 2. Crial Ambiente Virtial
No terminal PowerShell digite os seguintes comandos
>PS C:\...dentro_da_pasta_do_projeto> python -m venv venv

# 3. Ativar ambiente Virtual
Caso esteja utilizando o PowerShell será necessário alterar a politica de segurança no modo admnistrador com o seguinte comando:
>PS C:\Windows\system32> Set-ExecutionPolicy -Scope CurrentUser -ExecutionPolicy RemoteSigned

Reiniciar o terminal
>PS C:\\...\dentro_da_pasta_do_projeto> .\venv\Scripts\Activate.ps1

# 4. Instalar bibliotecas\modulos
Com o ambiente virtual ativo: digite os seguintes comandos
>(venv) PS C:\\...\dentro_da_pasta_do_projeto> <br>
pip install selenium <br>
pip install webdriver_manager <br>
pip install pywin32 <br>
pip install pandas <br>
pip install openpyxl <br>
pip install flask <br>

# Código webscrap

In [53]:
#importando brbliotecas
import time#Pausas no programa
from selenium.webdriver.common.by import By #Webscrap
from selenium import webdriver #Webscrap - Navegadores
from selenium.webdriver.chrome.service import Service #Webscrap
from selenium.webdriver.common.keys import Keys # Necessário para enviar caracteres especiais no comando .send_keys()
from webdriver_manager.chrome import ChromeDriverManager #Webscrap - Navegadores

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Definir site
Site = "https://nvd.nist.gov/vuln/search"

# Abrir o site
driver.get(Site)

### Funções
    
def  getListFromElements(Caminho):    
    '''
    Retorna uma lista com o texto dos elementos encontrados no caminho(xpath) 
    
    '''
    Elemento = driver.find_elements(By.XPATH , Caminho)
    Lista = []
    for item in Elemento:
        Lista.append(item.text)
    return Lista

def isLastPage():  
    '''
    Idendifica existência de página adicional e segue para próxima caso exista
    
    '''
    Navegador = driver.find_elements(By.XPATH, '//*[@id="row"]/div/div/nav/ul/li/a')
    Saida = True
    time.sleep(1) 
    for item in Navegador:
        if item.text == '>':
            print('Abrindo próxima página...')
            item.click()
            Saida = False
            break
        else:
            Saida = True
            
    return Saida   

def xlsxGenerate(Dict):
    '''
    Gera uma Planilha.xlsx a partir de um Dicionário
    '''
    return True

def colorize(text,color= "\033[0;0m"):
    '''
    Altera a cor do texto 'text' na cor 'color'
    '''
    text = color + text + colorReset
    return text

print('Declarando varáveis')    
colorReset = "\033[0;0m"
colorRed   = "\033[1;31m"
colorGreen = "\033[0;32m"
colorBlue  = "\033[1;34m"
colorCyan = "\033[1;36m"
fontBold    = "\033[;1m"
fontReverse = "\033[;7m"

Pesquisado = input('Software a ser pesquisado (T = teste): ')
if Pesquisado == 'T':
    Pesquisado = 'log4j'
    DataInicial = '01012022'
    DataFinal = '05012022'
else:
    DataInicial = input('Data inicial(mm/dd/aaaa-mmddaaaa): ')
    DataFinal = input('Data final(mm/dd/aaaa-mmddaaaa): ')

lstCVE = []
lstCVSS_v20 = []
lstCVSS_v31 = []
lstDescr = []
lstPubli = []
sistema = []
Links = []
Referencias = []
Configs = []

print('Definindo xpaths')
XpathCVE = '//*/tbody/tr/th/strong/a'
XpathCVSS_v20 = '//*/tbody/tr/td[2]/span[2]'
XpathCVSS_v31 = '//*/tbody/tr/td[2]/span[1]'
XpathDescr = '//*/tbody/tr/td[1]/p'
XpathPubli = '//*/table/tbody/tr/td[1]/span'
xpathReferencia = '//*/table/tbody/tr/td/div/div[1]/div/div[1]/table/tbody/tr/td[1]/a'
xpathConfig = '//*[@id="config-div-1"]/table/tbody/tr/td[1]/b'

print('Ativando busca avançada')
AdvavancedSearch = driver.find_element(value='SearchTypeAdvanced')
AdvavancedSearch.click()

print('Encontrando elementos na página')
Keyword = driver.find_element(value="Keywords")
Keyword.clear()

# Encontrar os elementos Data inicial e Data final
StartDate = driver.find_element(value='published-start-date')
StartDate.clear()

EndDate = driver.find_element(value='published-end-date')
EndDate.clear()

print('Inserindo configurações de busca')
Keyword.send_keys(Pesquisado)
StartDate.send_keys(DataInicial)
EndDate.send_keys(DataFinal)

print('Iniciando busca por vulnerabilidades em '+ colorize(Pesquisado,colorGreen))
# Clicar no botão "Search"
SubmitSearch = driver.find_element(value='vuln-search-submit')
SubmitSearch.click()
time.sleep(1)

LastPage = False        
while not(LastPage):
    lstCVE += getListFromElements(XpathCVE)
    lstCVSS_v20 += getListFromElements(XpathCVSS_v20)
    lstCVSS_v31 += getListFromElements(XpathCVSS_v31)
    lstDescr += getListFromElements(XpathDescr)
    lstPubli += getListFromElements(XpathPubli)
    LastPage = isLastPage()
    
if len(lstCVE) == 0:
    print('Na busca por ' + colorize(Pesquisado,colorBlue) + ', nenhuma vulnerabilidade foi encontrada')
elif len(lstCVE) == 1:
    print(len('>>> Uma nova vulnerabilidade encontrada'))
else:
    print('>>>', len(lstCVE), 'novas vulnerabilidades foram encontradas')   
        
for i in lstCVE:
    Links.append('https://nvd.nist.gov/vuln/detail/'+i)
    sistema.append(Pesquisado)
    
contador = len(Links)
print('Buscando informações de referencia e configuração para cada vulnerabilidade...')
for link in Links:
    driver.get(link)
    
    lstConfig = getListFromElements(xpathConfig)
    
    lstReferencia = getListFromElements(xpathReferencia)
    
    print(contador,'|',end=' ')
    Referencia = []    
    for r in range(len(lstReferencia)):
        Referencia.append(lstReferencia[r])
    Referencias.append(Referencia)
    
    Config = []    
    for l in range(len(lstConfig)):
        Config.append(lstConfig[l])
    Configs.append(Config)
    
    contador = contador - 1    
    
print('\nAdicionando informações ao dicionário')
Tabela={}
Tabela.update({'CVE': lstCVE})
Tabela.update({'CVSS_v20': lstCVSS_v20})
Tabela.update({'CVSS_v31': lstCVSS_v31})
Tabela.update({'Descrição': lstDescr})
Tabela.update({'Publicação': lstPubli})    
Tabela.update({'Link':Links})
Tabela.update({'Sistema':sistema})
Tabela.update({'Referencias':Referencias})
Tabela.update({'Configuração':Configs})

arquivo = open('relatório.txt', 'w')
arquivo.writelines(str(Tabela))
arquivo.close()

print("\nAqui está, tudo o que encontrei até agora:")
print(Tabela['Descrição'])
print('\nEncerrando o navegador.')
driver.quit()




====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\Charles Oliveira\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


Declarando varáveis
Software a ser pesquisado (T = teste): T
Definindo xpaths
Ativando busca avançada
Encontrando elementos na página
Inserindo configurações de busca
Iniciando busca por vulnerabilidades em log4j
>>> 9 novas vulnerabilidades foram encontradas
Buscando informações de referencia e configuração para cada vulnerabilidade...
9 | 8 | 7 | 6 | 5 | 4 | 3 | 2 | 1 | 
Adicionando informações ao dicionário

Aqui está, tudo o que encontrei até agora:
['Incomplete fix for CVE-2021-3100. The Apache Log4j hotpatch package starting with log4j-cve-2021-44228-hotpatch-1.1-16 will now explicitly mimic the Linux capabilities and cgroups of the target Java process that the hotpatch is applied to.', 'The Apache Log4j hotpatch package before log4j-cve-2021-44228-hotpatch-1.1-13 didn’t mimic the permissions of the JVM being patched, allowing it to escalate privileges.', 'GeoTools is an open source Java library that provides tools for geospatial data. The GeoTools library has a number of data so

# Criação da Planilha em XLSX

In [54]:


import pandas as pd # Para Criar Planilha
from openpyxl import Workbook, load_workbook # Para Formatar a Planilha
from openpyxl.styles import PatternFill, Font, Alignment # Para alterar os estilos de formatação das células

def planilha(Tabela):
    Tabela2 = Tabela.copy()
    """
    Cria uma planilha com os dados da 'Tabela'
    """    
    print('Iniciando verificação dos dados...')
    
    Tabela.get('CVSS_v20')
    cont = 0
    for i in Tabela.get('CVSS_v20'):
        _,valores_v2,_ = i.split(' ')
        
        Tabela2.get('CVSS_v20')[cont] = float(valores_v2)
        cont = cont+1

    print(Tabela.get('CVSS_v20'))
    cont=0
    for i in Tabela.get('CVSS_v31'):
        #print(i)
        _,valores_v3,_ = i.split(' ')
        #print(valores_v3)
        Tabela2.get('CVSS_v31')[cont] = float(valores_v3)
        cont = cont+1

    print(Tabela2.get('CVSS_v31'))

    df = pd.DataFrame.from_dict(Tabela, orient ='columns') #dicionario para dataframe
    df = df.rename(columns={'CVE_ID':'CVE','CVSS_v31':'Severity_v3','CVSS_v20':
                        'Severity_v2','Descrição':'Current Description','Publicação':'NVD','Sistema':'Software/Sistema','Referencias':'References','Configuração':'Known'}) #renomear as colunas no dataframe
    
    df = df[['Software/Sistema',
                   'CVE',
                   'Current Description',
                  'Severity_v2','Severity_v3',
                  'References',
                  'Known',
                  'NVD',
                  'Link']] #organizar a posição das colunas
    print('Conversão para planilha finalizada!')
    df.to_excel('Planilha.xlsx')
    print('Planilha criada com sucesso!')
    
    return df



def organizar(df):
    # Carregar dados para variável
    wb = load_workbook('Planilha.xlsx')
    # Escolhe active sheet
    ws = wb.active
    # Deleta primeira coluna, que é somente índice
    ws.delete_cols(1)
    # Cabeçalho em negrito e fundo azul
    # Fill parameters
    my_fill = PatternFill(start_color='5399FF', 
                       end_color='5399FF',
                       fill_type='solid')
    # Bold Parameter
    my_font = Font(bold=True) ##tamanho e fonte
    # Formata o cabeçalho
    my_header = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
    for cell in my_header:
        ws[cell].fill = my_fill
        ws[cell].font = my_font
    # Formata as colunas e linhas
    ws.column_dimensions['A'].width = 20
    ws.column_dimensions['B'].width = 20
    ws.column_dimensions['C'].width = 80
    ws.column_dimensions['D'].width = 20
    ws.column_dimensions['E'].width = 20
    ws.column_dimensions['F'].width = 40
    ws.column_dimensions['G'].width = 40
    ws.column_dimensions['H'].width = 40
    ws.column_dimensions['I'].width = 40
    for row in ws.iter_rows():  
        for cell in row:      
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    #alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    #cell = ws['C2']
    #ws.alignment = alignment
    # Salva o arquivo
    print('Planilha configurada')
    wb.save('Vulnerabilidades.xlsx')
    return 'Planilha salva'


Planilha = planilha(Tabela)
display(Planilha)

Planilha_formatada = organizar(Planilha)
display(Planilha_formatada)

Iniciando verificação dos dados...
[7.2, 7.2, 7.5, 7.5, 2.1, 9.0, 6.8, 6.0, 7.5]
[8.8, 8.8, 7.2, 9.8, 5.5, 8.8, 9.8, 8.8, 9.8]
Conversão para planilha finalizada!
Planilha criada com sucesso!


,Software/Sistema,CVE,Current Description,Severity_v2,Severity_v3,References,Known,NVD,Link
0,log4j,CVE-2022-0070,Incomplete fix for CVE-2021-3100. The Apache L...,7.2,8.8,[https://alas.aws.amazon.com/cve/html/CVE-2022...,[ cpe:2.3:a:amazon:hotpatch:*:*:*:*:*:log4j:*:...,"Abril 19, 2022; 7:15:13 PM -0400",https://nvd.nist.gov/vuln/detail/CVE-2022-0070
1,log4j,CVE-2021-3100,The Apache Log4j hotpatch package before log4j...,7.2,8.8,[https://alas.aws.amazon.com/AL2/ALAS-2021-173...,[ cpe:2.3:a:amazon:log4jhotpatch:*:*:*:*:*:*:*...,"Abril 19, 2022; 7:15:13 PM -0400",https://nvd.nist.gov/vuln/detail/CVE-2021-3100
2,log4j,CVE-2022-24818,GeoTools is an open source Java library that p...,7.5,7.2,[https://github.com/geotools/geotools/commit/4...,"[ cpe:2.3:a:geotools:geotools:*:*:*:*:*:*:*:*,...","Abril 13, 2022; 5:15:07 PM -0400",https://nvd.nist.gov/vuln/detail/CVE-2022-24818
3,log4j,CVE-2022-23848,"In Alluxio before 2.7.3, the logserver does no...",7.5,9.8,[https://www.alluxio.io/download/releases/allu...,[ cpe:2.3:a:alluxio:alluxio:*:*:*:*:*:*:*:*],"Fevereiro 20, 2022; 2:15:09 PM -0500",https://nvd.nist.gov/vuln/detail/CVE-2022-23848
4,log4j,CVE-2022-21704,log4js-node is a port of log4js to node.js. In...,2.1,5.5,[https://github.com/log4js-node/log4js-node/bl...,[ cpe:2.3:a:log4js_project:log4js:*:*:*:*:*:no...,"Janeiro 19, 2022; 6:15:08 PM -0500",https://nvd.nist.gov/vuln/detail/CVE-2022-21704
5,log4j,CVE-2022-23307,CVE-2020-9493 identified a deserialization iss...,9.0,8.8,[https://lists.apache.org/thread/rg4yyc89vs3dw...,"[ cpe:2.3:a:apache:chainsaw:*:*:*:*:*:*:*:*, ...","Janeiro 18, 2022; 11:15:08 AM -0500",https://nvd.nist.gov/vuln/detail/CVE-2022-23307
6,log4j,CVE-2022-23305,"By design, the JDBCAppender in Log4j 1.2.x acc...",6.8,9.8,[http://www.openwall.com/lists/oss-security/20...,[ cpe:2.3:a:apache:log4j:*:*:*:*:*:*:*:*],"Janeiro 18, 2022; 11:15:08 AM -0500",https://nvd.nist.gov/vuln/detail/CVE-2022-23305
7,log4j,CVE-2022-23302,JMSSink in all versions of Log4j 1.x is vulner...,6.0,8.8,[http://www.openwall.com/lists/oss-security/20...,[ cpe:2.3:a:apache:log4j:*:*:*:*:*:*:*:*],"Janeiro 18, 2022; 11:15:08 AM -0500",https://nvd.nist.gov/vuln/detail/CVE-2022-23302
8,log4j,CVE-2021-44530,An injection vulnerability exists in a third-p...,7.5,9.8,[https://community.ui.com/releases/Security-Ad...,[ cpe:2.3:a:ui:unifi_network_controller:*:*:*:...,"Janeiro 14, 2022; 3:15:12 PM -0500",https://nvd.nist.gov/vuln/detail/CVE-2021-44530


Planilha configurada


'Planilha salva'

# Envio automático de E-mail

In [56]:
#!pip install pywin32
import win32com.client as win32 # importar biblioteca para criar integraçãao entre o Python e o Outlook
import os,sys #auxilia na validação de Pathway para diretorio
import re #auxilia na validação de email
from datetime import date #auxilia no trabalho de datas

parametro= 7
def valores(Tabela, parametro= 7):
    """
    Filtra 'Tabela' e retorna lista com Vulnerabilidades consideradas críticas (acima do 'parametro')
    """
    envio_2 = len([i for i in Tabela.get('CVSS_v20') if i > 8.0])
    print('Vulnerabilidades acima de', parametro,'encontradas [v2.0]: ' + str(envio_2))
    envio_3 = len([i for i in Tabela.get('CVSS_v31') if i > 8.0])
    print('Vulnerabilidades acima de', parametro,'encontradas [v3.1]: ' + str(envio_3))
    lista = []
    cont=0
    for i in Tabela.get('CVSS_v20'):
        if i > 8.0:
            lista.append(Tabela.get('CVE')[cont])
            lista.append(Tabela.get('CVSS_v20')[cont])
            lista.append(Tabela.get('CVSS_v31')[cont])
            lista.append(Tabela.get('Descrição')[cont])
            lista.append(Tabela.get('Publicação')[cont])    
            lista.append(Tabela.get('Link')[cont])
            lista.append(Tabela.get('Sistema')[cont])
            lista.append(Tabela.get('Referencias')[cont])
            lista.append(Tabela.get('Configuração')[cont])
        cont=cont+1
    cont=0
    for i in Tabela.get('CVSS_v31'):
        if i > 8.0:
            lista.append(Tabela.get('CVE')[cont])
            lista.append(Tabela.get('CVSS_v20')[cont])
            lista.append(Tabela.get('CVSS_v31')[cont])
            lista.append(Tabela.get('Descrição')[cont])
            lista.append(Tabela.get('Publicação')[cont])    
            lista.append(Tabela.get('Link')[cont])
            lista.append(Tabela.get('Sistema')[cont])
            lista.append(Tabela.get('Referencias')[cont])
            lista.append(Tabela.get('Configuração')[cont])
        cont=cont+1
    #print(lista)
    return(lista,envio_3)

def valores(Tabela, parametro= 7):
    """
    Filtra 'Tabela' e retorna lista com Vulnerabilidades consideradas críticas (acima do 'parametro')
    """
    envio_2 = len([i for i in Tabela.get('CVSS_v20') if i > 8.0])
    print('Vulnerabilidades acima de', parametro,'encontradas [v2.0]: ' + str(envio_2))
    envio_3 = len([i for i in Tabela.get('CVSS_v31') if i > 8.0])
    print('Vulnerabilidades acima de', parametro,'encontradas [v3.1]: ' + str(envio_3))
    lista = []
    cont=0
    for i in Tabela.get('CVSS_v20'):
        if i > 8.0:
            lista.append(Tabela.get('CVE')[cont])
            lista.append(Tabela.get('CVSS_v20')[cont])
            lista.append(Tabela.get('CVSS_v31')[cont])
            lista.append(Tabela.get('Descrição')[cont])
            lista.append(Tabela.get('Publicação')[cont])    
            lista.append(Tabela.get('Link')[cont])
            lista.append(Tabela.get('Sistema')[cont])
            lista.append(Tabela.get('Referencias')[cont])
            lista.append(Tabela.get('Configuração')[cont])
        cont=cont+1
    cont=0
    for i in Tabela.get('CVSS_v31'):
        if i > 8.0:
            lista.append(Tabela.get('CVE')[cont])
            lista.append(Tabela.get('CVSS_v20')[cont])
            lista.append(Tabela.get('CVSS_v31')[cont])
            lista.append(Tabela.get('Descrição')[cont])
            lista.append(Tabela.get('Publicação')[cont])    
            lista.append(Tabela.get('Link')[cont])
            lista.append(Tabela.get('Sistema')[cont])
            lista.append(Tabela.get('Referencias')[cont])
            lista.append(Tabela.get('Configuração')[cont])
        cont=cont+1
    #print(lista)
    return(lista,envio_3)

def diretorio():
    path = os.getcwd()
    file_path = path + "\Vulnerabilidades.xlsx"
    return file_path

def validar_nome(nome):                     #define a função validar_nome    
    if " " in nome:                         #Se o nome não possuir espaçamento, considera como inválido
        [p_nome,s_nome]= nome.split()       #divide a variável nome em p_nome (primeiro nome) e s_nome (sobrenome)
        p = len(p_nome)                     #variável p recebe a quantidade de caracteres do primeiro nome
        s = len(s_nome)                     #variável s recebe a quantidade de caracteres do sobrenome
        if (p > 1 and s > 1):               #se p e s possuirem mais que uma letra, valida o nome
            nome_certo = True               #variável recebe True por cumprir os requisitos, caso contrário recebe False
        else:
            nome_certo = False
    else:
        nome_certo = False
    
    return nome_certo                       #retorna nome_certo


def validar_email(destinatario):              #define a função validar_email    
    email_certo = re.search(r'[a-zA-Z0-9_-]+@[a-zA-Z0-9]+.[a-zA-Z]{1,3}$', destinatario)
    print(str(email_certo))
    
    if email_certo:
        return True
    else:
        return False
    
def Enviar_Email(lista, val, nome, destinatario, anexo_endereco='Vulnerabilidades.xlsx'):# valor menor que parametro    
    outlook = win32.Dispatch('outlook.application') # Cria a integração com o Outlook
    email = outlook.CreateItem(0) #Gera um email
    email.To = destinatario # define o endereço de email de destino

    #Data de atualização do registro de vulnerabilidade    
    from datetime import datetime
    atualizacao = datetime.now()
    atualizacao = atualizacao.strftime("%d/%m/%Y %H:%M")
    
    #Valores transmitidos para a mensagem de texto. O valor de cada variável virá da planilha de vulnerabilidade
    Software = lista
    CVE = lista
    Severity = lista
    Occurrence = lista
    Link = lista
    
    # Anexo do E-mail
    anexo = anexo_endereco   #"C://Users/aluno/WebScraping/teste.xlsx"
    email.Attachments.Add(anexo)   #anexa ao email o arquivo solicitado
    
    # Assunto do E-mail    
    email.Subject = "Assunto: Vulnerabilidades encontradas" # define o assunto do email   
    
    # Corpo do E-mail
    email.HTMLBody = f""" 
    
    
       
    <p>Prezado, {nome}. Segue atualização sobre vulnerabilidades acima com severidade acima de {parametro} pontos.</p>
    
    <h1> Vulnerabilidades Encontradas </h1>
    
    
    <!--
        HIERARQUIA DE TABELAS (simples)
        TABLE = tabela
            TABLE ROW = linha de tabela
                TABLE HEADER = cabeçalho de tabela
                TABLE DATA = dado de tabela
                
    -->
    <table border = "1" border-collapse = "0" class = "dataframe>
        <thead> <!-- Cabeçalho -->
        
            <tr style = "text-align: center"> 
                <th>Software</th>
                <th>CVE</th>
                <th>Severity</th>
                <th>NVD Published Date</th>
                <th>Link para o respectivo CVE</th>
            </tr>
        </thead>
        <tbody>    
        
        """
    valores = 0
    for i in range(val):
        email.HTMLBody = email.HTMLBody + f"""
            <tr style = "text-align: right"> <!-- 1º linha -->
                <td>{Software[6+(valores)]}</td>
                <td>{CVE[0+(valores)]}</td>
                <td>{Severity[2+(valores)]}</td>
                <td>{Occurrence[4+(valores)]}</td>
                <td>{Link[5+(valores)]}</td> 
            </tr>
            
        </tbody>
        """
        valores=valores+9
    email.HTMLBody = email.HTMLBody + f"""
    </table>
            
    <p>As vulnerabilidades foram atualizadas em: {atualizacao} </p>

    <p>atenciosamente, </p>


    """ 
    #Corpo do email. O comando HTMLBody ao invés de criar apenas um texto, gera um template de email utilizando linguagem HTML.
    #Com isso, permite várias configurações de páginas como acrescentar imagens, arquivos e outros itens
    #Para criar um parágrafo em HTML, a linha precisa iniciar como <p>, e finalizar com </p>
    # o comando (f""" texto de email""") torna dinâmico o texto do email, possibilitando acrescentar variáveis calculadas ao longo do código
    print(email.HTMLBody)
    #email.display()
    email.Send()
    
    
# início de solução Web Scraping

email_certo = False   #iniciar uma variável que receberá a mensagem de validação de e-mail
nome_certo = False    #iniciar uma variável que receberá a mensagem de validação de nome

while nome_certo != True:                           #enquanto a variavel "valida_nome" não for verdadeiro, o código solicita para o usuário escrever um nome correto
    nome = input("Digite o seu nome e sobrenome: ") #usuário digita nome para validação
    nome_certo = validar_nome(nome)                 #Chama a função validar_nome

while email_certo != True:                        #enquanto a variavel "valida_email" não for verdadeiro, o código solicita para o usuário escrever um e-mail correto
    destinatario = input("Digite o seu e-mail: ") #usuário digita e-mail para validação
    email_certo = validar_email(destinatario)     #Chama a função validar_email
    
lista,val = valores(Tabela)
anexo_endereco = diretorio() 

Enviar_Email(lista, val, nome, destinatario, anexo_endereco= anexo_endereco)

print("email eviado")

Digite o seu nome e sobrenome: Charles Oliveira
Digite o seu e-mail: neo_charles@outlook.com
<re.Match object; span=(0, 23), match='neo_charles@outlook.com'>
Vulnerabilidades acima de 7 encontradas [v2.0]: 1
Vulnerabilidades acima de 7 encontradas [v3.1]: 7
 
    
    
       
    <p>Prezado, Charles Oliveira. Segue atualização sobre vulnerabilidades acima com severidade acima de 7 pontos.</p>
    
    <h1> Vulnerabilidades Encontradas </h1>
    
    
    <!--
        HIERARQUIA DE TABELAS (simples)
        TABLE = tabela
            TABLE ROW = linha de tabela
                TABLE HEADER = cabeçalho de tabela
                TABLE DATA = dado de tabela
                
    -->
    <table border = "1" border-collapse = "0" class = "dataframe>
        <thead> <!-- Cabeçalho -->
        
            <tr style = "text-align: center"> 
                <th>Software</th>
                <th>CVE</th>
                <th>Severity</th>
                <th>NVD Published Date</th>
              